In [2]:
import sys
import numpy as np
import string

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Bidirectional

LAYER_COUNT = 3
HIDDEN_LAYERS_DIM = 500

Using TensorFlow backend.


In [3]:
# generic vocabulary
characters = list(string.printable)
characters.remove('\x0b')
characters.remove('\x0c')

VOCABULARY_SIZE = len(characters)
characters_to_ix = {c:i for i,c in enumerate(characters)}
print("vocabulary len = %d" % VOCABULARY_SIZE)
print(characters)

vocabulary len = 98
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ', '\t', '\n', '\r']


In [7]:
test_model = Sequential()
for i in range(LAYER_COUNT):
    test_model.add(
            Bidirectional(
                LSTM(
                    HIDDEN_LAYERS_DIM, 
                    return_sequences=True,
                    stateful=True
                ),
            batch_input_shape=(1, 1, VOCABULARY_SIZE)
            )
        )
test_model.add(Dense(VOCABULARY_SIZE))
test_model.add(Activation('softmax'))
test_model.compile(loss='categorical_crossentropy', optimizer="adam")

In [12]:
test_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (1, 1, 1000)              2396000   
_________________________________________________________________
bidirectional_5 (Bidirection (1, 1, 1000)              6004000   
_________________________________________________________________
bidirectional_6 (Bidirection (1, 1, 1000)              6004000   
_________________________________________________________________
dense_2 (Dense)              (1, 1, 98)                98098     
_________________________________________________________________
activation_2 (Activation)    (1, 1, 98)                0         
Total params: 14,502,098
Trainable params: 14,502,098
Non-trainable params: 0
_________________________________________________________________


In [10]:
ls

4-gpu_BS-100_3-500_dp0.20_60S_epoch01-loss1.7700-val-loss1.3981_weights
4-gpu_BS-100_3-500_dp0.20_60S_epoch01-loss2.9326-val-loss2.4937_weights
4-gpu_BS-80_3-500_dp0.20_60S_epoch01-loss1.4302-val-loss1.2292_weights
bash_hp.sh
bash_hp_bi.sh
bi-4-gpu_bi_BS-100_3-500_dp0.20_60S_epoch{epoch01}-loss{loss:.4f}-val-loss{val_loss:.4f}_weights
data/
harry_potter_test-Copy1.ipynb
harry_potter_test.ipynb
harry_potter_train.ipynb
harry_potter_train.py
harry_potter_train_bi.py
rnn_lstm_v1/


In [11]:
test_model.load_weights(
    "bi-4-gpu_bi_BS-100_3-500_dp0.20_60S_epoch{epoch01}-loss{loss:.4f}-val-loss{val_loss:.4f}_weights"
)

In [18]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array"""
    # from fchollet/keras
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def predict_next_char(model, current_char, diversity=1.0):
    """Predict the next character from the current one"""
    x = np.zeros((1, 1, VOCABULARY_SIZE))
    x[:,:,characters_to_ix[current_char]] = 1
    y = model.predict(x, batch_size=1)
    next_char_ix = sample(y[0,:], temperature=diversity)
    next_char = characters[next_char_ix]
    return next_char

def generate_text(model, seed="I am", count=80):
    """Generate characters from a given seed"""
    model.reset_states()
    for s in seed[:-1]:
        next_char = predict_next_char(model, s)
    current_char = seed[-1]

    sys.stdout.write("["+seed+"]")
    
    for i in range(count - len(seed)):
        next_char = predict_next_char(model, current_char, diversity=0.5)
        current_char = next_char
        sys.stdout.write(next_char)
    print("...\n")

In [19]:
sentence = "I wanted to try to get the potion. But this is not working."

In [20]:
for i in range(5):
    generate_text(
        test_model,
        seed=sentence
    )

ValueError: object too deep for desired array